In [1]:
from PIL import Image

In [2]:
def hide_message(image_path, message):
    # open the image file
    img = Image.open(image_path)

    # convert the message to binary format
    binary_message = ''.join(format(ord(i), '08b') for i in message)

    # get the size of the image
    width, height = img.size

    # calculate the maximum number of bits that can be hidden in the image
    max_bits = width * height * 3 // 8

    # check if the message is too long to be hidden in the image
    if len(binary_message) > max_bits:
        raise ValueError("Message is too long to be hidden in the image")

    # add a stop bit to the binary message
    binary_message += '0' * (max_bits - len(binary_message) - 1) + '1'

    # create a new image object to hold the modified image
    steg_img = Image.new(img.mode, img.size)

    # iterate over each pixel in the image
    for x in range(width):
        for y in range(height):
            # get the RGB values for the current pixel
            r, g, b = img.getpixel((x, y))

            # get the next bit from the binary message
            bit = int(binary_message[0])

            # hide the bit in the least significant bit of the red, green, or blue value
            if bit == 0:
                r = r & ~1
            else:
                r = r | 1

            # remove the first bit from the binary message
            binary_message = binary_message[1:]

            # set the pixel in the steganographic image to the modified RGB values
            steg_img.putpixel((x, y), (r, g, b))

    # save the steganographic image to a new file
    steg_img.save("steg_" + image_path)
    print("Message hidden in the image")



In [3]:
def extract_message(image_path):
    # open the steganographic image file
    steg_img = Image.open('/content/Sample.jpeg')

    # get the size of the image
    width, height = steg_img.size

    # create a string to hold the extracted binary message
    binary_message = ''

    # iterate over each pixel in the image
    for x in range(width):
        for y in range(height):
            # get the RGB values for the current pixel
            r, g, b = steg_img.getpixel((x, y))

            # extract the least significant bit from the red, green, or blue value
            bit = r & 1

            # add the bit to the binary message
            binary_message += str(bit)

            # check if the binary message ends with the stop bit
            if binary_message[-1] == '1':
                # convert the binary message to a string and return it
                message = ''
                for i in range(0, len(binary_message) - 1, 8):
                    message += chr(int(binary_message[i:i+8], 2))
                return message

    raise ValueError("No message found in the image")


In [5]:
# Example usage
image_path = "/content/Sample.jpeg"
message = "This is a secret message!"
#hide_message(image_path, message)
extracted_message = extract_message("steg_" + image_path)
print("Extracted message:", extracted_message)

Extracted message: 
